<a href="https://colab.research.google.com/github/jutrera/InnoSoft-2018/blob/master/InnoSoft_2018_Convolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# InnoSoft 2018 - Convolutional nets

## Preparamos el entorno

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

In [0]:
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive 
!google-drive-ocamlfuse drive

In [0]:
print('Files in Drive:')
!ls drive/

## Obtenemos las librerías

In [0]:
import numpy as np
from scipy import misc
from PIL import Image
import glob
import matplotlib.pyplot as plt
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline
from IPython.display import SVG
import cv2
import seaborn as sn
import pandas as pd
import pickle
from keras import layers
from keras.layers import Flatten, Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout
from keras.models import Sequential, Model, load_model
from keras.preprocessing import image
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.imagenet_utils import decode_predictions
from keras.utils import layer_utils, np_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
from keras import losses
from keras import regularizers
import keras.backend as K
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf

In [0]:
!pip install keras-metrics
import keras_metrics

##Preparamos el conjunto de datos

In [0]:
from keras.datasets import cifar100

(x_train_original, y_train_original), (x_test_original, y_test_original) = cifar100.load_data(label_mode='fine')

In [0]:
imgplot = plt.imshow(x_train_original[0])
plt.show()

In [0]:
y_train = np_utils.to_categorical(y_train_original, 100)
y_test = np_utils.to_categorical(y_test_original, 100)

x_train = x_train_original/255
x_test = x_test_original/255

##Inicializamos Keras

In [0]:
K.set_learning_phase(1)
K.set_image_data_format('channels_last')

##Definimos la red convolutiva

La red parte del conjunto de imágenes de 32x32x3. Queremos hacer lo siguiente:



1.   Convolución de 32 kernels en ventanas de 3x3, con padding inicial de 2x2.
2.   Convolución de 64 kernels en ventanas de 3x3, sin padding ni stride
3.   Max Pooling en ventanas de 2x2
4.   Convolución de 128 kernels en ventanas de 3x3.
5.   Convolución de 256 kernels en ventanas de 3x3.
6.   Convolución de 512 kernels de 3x3 y stride 2
7.   Max Pooling de 2x2
8.   Convolución de 1024 kernels de 3x3 y stride 2
9.   Max Pooling de 2x2
10. Flatten (conversión en array de dimensión 1)

Para la función de activación se usará **ReLu**

**Pistas**:
*   Ver [Documentación Keras: Conv2D](https://keras.io/layers/convolutional/) para Padding y Convolución
*   Ver [Documentación Keras: Pooling](https://keras.io/layers/pooling/) para Pooling
*   Ver [Documentación Keras: Flatten](https://keras.io/layers/core/) para Flatten







In [0]:
def create_cnn():
  model = Sequential()
  #Code here
  
  #Code here
  model.add(Dense(500, activation='relu'))
  model.add(Dropout(0.5)) #Add Dropout to 
  model.add(Dense(100, activation='softmax'))
  
  return model

## Entrenamos la red creada

In [0]:
ConvNN = create_cnn()
ConvNN.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['acc', 'mse'])

In [0]:
ConvNN.summary()

In [0]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_delta=1e-4)

cnn_history = ConvNN.fit(x=x_train, y=y_train, batch_size=32, epochs=20, verbose=1, validation_data=(x_test, y_test), shuffle=True, callbacks=[reduce_lr])

##Vemos los resultados

In [0]:
plt.figure(0)
plt.plot(cnn_history.history['acc'],'r')
plt.plot(cnn_history.history['val_acc'],'g')
plt.xticks(np.arange(0, 15, 1.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Accuracy")
plt.title("Training Accuracy vs Validation Accuracy")
plt.legend(['train','validation'])
 
plt.figure(1)
plt.plot(cnn_history.history['loss'],'r')
plt.plot(cnn_history.history['val_loss'],'g')
plt.xticks(np.arange(0, 15, 1.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Loss")
plt.title("Training Loss vs Validation Loss")
plt.legend(['train','validation'])

plt.figure(2)
plt.plot(cnn_history.history['mean_squared_error'],'r')
plt.plot(cnn_history.history['val_mean_squared_error'],'g')
plt.xticks(np.arange(0, 15, 1.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("MSE")
plt.title("Training Loss vs Validation Loss")
plt.legend(['train','validation'])
 
plt.show()

In [0]:
ConvNN.save('drive/InnoSoft 2018/ConvNN_Model.h5')

In [0]:
evaluation = ConvNN.evaluate(x=x_test, y=y_test, batch_size=32, verbose=1)
print(evaluation)

In [0]:
from sklearn.metrics import confusion_matrix, classification_report

cnn_pred = ConvNN.predict(x_test, batch_size=32, verbose=1)
cnn_predicted = np.argmax(cnn_pred, axis=1)

In [0]:
print('Confusion Matrix')
cm = confusion_matrix(np.argmax(y_test, axis=1), cnn_predicted)
plt.figure(figsize = (30,20))
sn.set(font_scale=1.4) #for label size
sn.heatmap(cm, annot=True, annot_kws={"size": 12}) # font size
plt.show()
print()
print('Classification Report')
print(classification_report(np.argmax(y_test, axis=1), cnn_predicted))

In [0]:
from sklearn.datasets import make_classification
from sklearn.preprocessing import label_binarize
from scipy import interp
from itertools import cycle

n_classes = 100

from sklearn.metrics import roc_curve, auc

# Plot linewidth.
lw = 2

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], cnn_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), cnn_pred.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure(1)
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(10), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()